In [1]:
import pathlib
import random
import pandas as pd
import numpy as np
import sys

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict

from sklearn.metrics import (
    f1_score, 
    accuracy_score,
    classification_report, 
)

ROOT_DIR = pathlib.Path().absolute()
DATA_DIR = ROOT_DIR / "data"
RANDOM_SEED = 42

## Загрузка и обзор данных

In [3]:
df_trends = pd.read_csv(DATA_DIR / "trends_description.csv")
df = pd.read_csv(DATA_DIR / "train.csv")

In [4]:
df.head()

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Обучение моделей

### Предобработка данных

In [5]:
df.head()

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X, y = df[["text"]], df[[f"trend_id_res{i}" for i in range(50)]]
X = X.astype("str").copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_SEED)
print(f"X_train.shape is {X_train.shape}")
print(f"y_train.shape is {y_train.shape}")
print(f"X_test.shape is {X_test.shape}")
print(f"y_test.shape is {y_test.shape}")

X_train.shape is (3698, 1)
y_train.shape is (3698, 50)
X_test.shape is (925, 1)
y_test.shape is (925, 50)


In [8]:
X_train

,text
1538,"Ну, за [NUM]ч. и [NUM] мин. мне ещё никогда не..."
2991,Доставка всегда осуществляется значительно дол...
2812,Задержка доставки
4515,"Отличный сервис, только бы ассортимент расшири..."
4531,"Поддержка говно, курьеры опаздывают минут на [..."
...,...
4426,"+ быстро. - иногда сумма заказа очень велика, ..."
466,🦉
3092,До самоката я тратил меньше денег в день
3772,О вас редко думаю. Напрягает СТМ. Когда непоня...


###  Проверка качества на тречнировчном датасете

In [39]:
preprocessor = ColumnTransformer(
    [
        ("vetorizer", TfidfVectorizer(analyzer="char_wb", ngram_range = (1,3)), "text")
    ],                         
    remainder = "passthrough"
)

pipeline_multiout = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clf", MultiOutputClassifier(LogisticRegression(max_iter = 10_000))),
    ]
)
display(pipeline_multiout)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vetorizer',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  ngram_range=(1,
                                                                               3)),
                                                  'text')])),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=10000)))])

In [20]:
cross_valid = cross_validate(pipeline_multiout, 
                            #  X_train, y_train, 
                             X, y,
                             cv = 5, scoring = ["accuracy"], n_jobs = -1)
print("test_accuracy:", cross_valid["test_accuracy"].mean())

test_accuracy: 0.26648859248859247


In [24]:
model = pipeline_multiout.fit(X_train, y_train)

In [32]:
model.predict(X_test)[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [28]:
model.predict_proba(X_test)

[array([[0.89425895, 0.10574105],
        [0.98754924, 0.01245076],
        [0.81664122, 0.18335878],
        ...,
        [0.83558201, 0.16441799],
        [0.96098866, 0.03901134],
        [0.95326749, 0.04673251]]),
 array([[0.95119411, 0.04880589],
        [0.75815679, 0.24184321],
        [0.99748105, 0.00251895],
        ...,
        [0.82175914, 0.17824086],
        [0.99352892, 0.00647108],
        [0.97185591, 0.02814409]]),
 array([[0.93901227, 0.06098773],
        [0.93636677, 0.06363323],
        [0.99330508, 0.00669492],
        ...,
        [0.74127215, 0.25872785],
        [0.98429147, 0.01570853],
        [0.97338091, 0.02661909]]),
 array([[0.9712444 , 0.0287556 ],
        [0.95622887, 0.04377113],
        [0.98015317, 0.01984683],
        ...,
        [0.8914542 , 0.1085458 ],
        [0.97165394, 0.02834606],
        [0.96664661, 0.03335339]]),
 array([[0.97748532, 0.02251468],
        [0.97092366, 0.02907634],
        [0.99465214, 0.00534786],
        ...,
        [

In [19]:
preprocessor.fit_transform(X_train)[0]#.todense()

<1x9285 sparse matrix of type '<class 'numpy.float64'>'
	with 177 stored elements in Compressed Sparse Row format>

In [33]:
y_pred = cross_val_predict(pipeline_multiout, X_train, y_train, cv = 2)

In [36]:
# Посмотрим на различные метрики
print(classification_report(y_train, y_pred, zero_division = 0))

              precision    recall  f1-score   support

           0       0.87      0.39      0.54       661
           1       0.89      0.09      0.16       270
           2       0.80      0.37      0.50       486
           3       0.93      0.21      0.35       289
           4       0.00      0.00      0.00       108
           5       0.00      0.00      0.00        44
           6       0.00      0.00      0.00        16
           7       0.00      0.00      0.00        27
           8       1.00      0.01      0.02       109
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        76
          11       0.00      0.00      0.00        87
          12       0.97      0.41      0.57       491
          13       0.00      0.00      0.00        29
          14       0.00      0.00      0.00        62
          15       0.00      0.00      0.00        66
          16       0.00      0.00      0.00       166
          17       0.00    

In [37]:
# Посмотрим на целевую метрику
accuracy_score(y_train, y_pred)

0.19334775554353706

###  Тренировка окончательной модели

In [40]:
pipeline_multiout.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vetorizer',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  ngram_range=(1,
                                                                               3)),
                                                  'text')])),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=10000)))])

##  Предсказание и загрузка решения

In [41]:
test =  pd.read_csv(DATA_DIR / "test.csv")

In [42]:
pred_test = pipeline_multiout.predict(test[["text"]].astype("str"))

In [194]:
pred_test[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [199]:
np.where(pred_test[1] == 1)[0].astype(str)

array(['12'], dtype='<U21')

In [229]:
answer = []
np.apply_along_axis(
    lambda x: answer.append(' '.join(np.where(x == 1)[0].astype(str))),
    axis=1,
    arr=pred_test
)

array([None, None, None, ..., None, None, None], dtype=object)

In [230]:
res = test[["index"]]
res['target'] = answer

/var/folders/3r/wlt6fvhn59s4zlkrgdbsymn00000gq/T/ipykernel_71182/879400367.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['target'] = answer


In [231]:
res

,index,target
0,3135,
1,4655,12
2,22118,2
3,23511,0
4,45,
...,...,...
9010,3523,3
9011,24925,
9012,6327,
9013,530,


In [160]:
# (
#     res.melt(
#         id_vars = "index", 
#         value_vars = [f"trend_id_res{i}" for i in range(50)], 
#         var_name = "trend_id", 
#         value_name = "target"
#     )
#     .groupby('index', sort=False)
#     .sum('target')
#     .assign(
#         target=lambda x: x['target'].astype(str).replace('0', '')
#     )
#     .reset_index()
#     .to_csv(DATA_DIR / "my_baseline_submission.csv", index=False)
# )

In [232]:
res.to_csv(DATA_DIR / "my_baseline_submission.csv", index=False)